In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir('/Users/zhenyamordan/PyCharmProjects/Kinome-Regularization/')

In [3]:
import pandas as pd
import plotly.express as px
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

In [4]:
!ls

CV_utils.py                   graph_utils.py
Linreg_clean.ipynb            model_sanity_check.ipynb
README.md                     phenotype_preprocess.py
RegressionModels.py           phenotype_preprocessing.ipynb
fof                           plot_utils.py


In [7]:
base_path = './fof/data/r/hyperparameter_tuning_cross_validation/'

In [8]:
experiments = [i for i in os.listdir(base_path) if '.DS_Store' not in i]
experiments

['v0_0.1_Kp_3_Kt_7_A_0.1_B_0.1', 'v0_0.1_Kp_2_Kt_7_A_0.1_B_0.1']

In [9]:
def parse_hyperparameters(experiment):
    split_elements = experiment.split('_')
    return ', '.join([f'{split_elements[i]}: {split_elements[i + 1]}' for i in range(0, len(split_elements), 2)])

In [10]:
parse_hyperparameters(experiments[0])

'v0: 0.1, Kp: 3, Kt: 7, A: 0.1, B: 0.1'

In [19]:
def plot_results(base_path, experiments):
    aggregated_result = {}
    for i, experiment in enumerate(experiments):
        formatted_hyperparameters = parse_hyperparameters(experiment)
        print(i, formatted_hyperparameters)
        folder_path = base_path + f'{experiment}/'
        plots_folder_path = base_path + f'{experiment}/plots/'
        
        os.makedirs(folder_path, exist_ok=True)
        os.makedirs(plots_folder_path, exist_ok=True)

        y_test_predicted = pd.read_csv(folder_path + f'y_test_predicted_final.csv')['x'].to_numpy()
        y_test_true_standartized = pd.read_csv(folder_path + f'y_test_true_standartized_final.csv')['x'].to_numpy()
        
        aggregated_result[experiment] = {
            'test_mse': mean_squared_error(y_test_true_standartized, y_test_predicted),
            'test_r2': r2_score(y_test_true_standartized, y_test_predicted)
        }
        print('Test MSE\t', aggregated_result[experiment]['test_mse'])
        print('Test R2\t', aggregated_result[experiment]['test_r2'])
        
        fig = px.scatter(x=y_test_predicted, y=y_test_true_standartized, title=f'Test, {formatted_hyperparameters}')
        fig.update_xaxes(title_text="Predicted Values")  # X-axis label
        fig.update_yaxes(title_text="True Standarized Values")  # Y-axis label
        fig.add_shape(
            type='line',
            x0=min(y_test_true_standartized),
            y0=min(y_test_true_standartized),
            x1=max(y_test_true_standartized),
            y1=max(y_test_true_standartized),
            line=dict(color='red', width=2, dash='dash'),
        )
        fig.show()
        fig.write_html(folder_path + 'true_predicted_scatter_plot_test.html')   
    return aggregated_result

In [20]:
aggregated_result = plot_results(base_path, experiments)

0 v0: 0.1, Kp: 3, Kt: 7, A: 0.1, B: 0.1
Test MSE	 438.074481511404
Test R2	 -0.6229150830810743


1 v0: 0.1, Kp: 2, Kt: 7, A: 0.1, B: 0.1
Test MSE	 438.07732357841036
Test R2	 -0.6229256119605378


In [67]:
result_df = pd.DataFrame(aggregated_result)
result_df = result_df.rename(columns = {i: parse_hyperparameters(i) for i in result_df.columns})

### Aggregated results overview

In [68]:
px.bar(result_df, barmode='group')

### Aggregated results

In [69]:
for i, row in result_df.iterrows():
    fig = px.bar(row, barmode='group')
    fig.show()